In [5]:
import ee
import geemap
from utils import import_data

# Authenticate to Earth Engine
try:
  ee.Initialize()
except Exception as e:
  ee.Authenticate()
  ee.Initialize(project='ee-ana-zonia')

import_data('br_amazon')

NameError: name 'ee' is not defined

In [2]:
import os
os.getcwd()

'C:\\Users\\anaca\\Desktop\\forest_regrowth\\scripts'

## Ages

Rewriting the code for "Benchmark maps of 33 years of secondary forest age for Brazil" to run with Collection 8 and include only class 3.

In [20]:
''' Making Masks
Select pixels used to determine secondary forest ages, and excluding unwanted ones from the calculation

We exclude:
- water bodies (possible misclassifications)
- urban areas
'''

# INDEX for land use types we are interested in
# 15 = pasture
# 39 = soy
# 20 = sugar cane
# 40 = rice
# 62 = cotton
# 41 = other temporary crop
# 46 = coffee
# 47 = citrus
# 35 = palm oil
# 48 = other perennial crop
# 9 = forest plantation
LU_index = [15, 39, 20, 40, 62, 41, 46, 47, 35, 48, 9]
lulc_palette = ["#f1c232", "#FFFFB2", "#FFD966", "#E974ED", "#D5A6BD", \
                "#e075ad", "#C27BA0", "#982c9e", "#e787f8", "#cd49e4", "#ad4413"]

# Create a binary mask with all pixels that have one of the anthropic land use classes we listed above
# creates an anthropic_mask image with one band per year
anthropic_mask = ee.Image().byte()
for i in range(first_year, last_year + 1):
    year = 'classification_' + str(i)
    # replaces all values in LU_index list with 1 to create binary mask
    anthropic = lulc.select(year).remap(LU_index, [1] * len(LU_index), 0).rename(year)
    anthropic_mask = anthropic_mask.addBands(anthropic)
anthropic_mask = anthropic_mask.select(anthropic_mask.bandNames().slice(1))

# Create a binary mask for non-water bodies
w_mask  = ee.Image("JRC/GSW1_4/GlobalSurfaceWater").select("max_extent").clip(roi).remap([0,1],[1,0]);

# Create a binary mask for non-urban areas
urban = ee.Image("DLR/WSF/WSF2015/v1").clip(roi)
# This will invert the mask, city pixels will be False (or 0) and non-urban pixels will be True (or 1)
inverse_mask = urban.eq(1).Not()
urban = urban.updateMask(inverse_mask)  # This will replace 1 values (city pixels) with NA
urban_mask = urban.unmask(1)  # This will replace NA values (non-urban pixels) with 1

In [21]:
'''
### Step 1 - Select Forested Areas Only

Take only pixels classified as "forest formation" (class 3) and "flooded forest" (class 6) from MapBiomas land use Collection 8.

Intakes:
    lulc -> MapBiomas Collection 8 for Land Use Land Cover raw data

Returns:
    mapbiomas_forest -> Image with one band per year (1985 - 2020) and a binary mask for FORESTED PIXELS
'''

mapbiomas_forest = ee.Image()

for i in range(first_year, last_year+1):
    year = 'classification_' + str(i)
    forest = lulc.select(year)
    forest = forest.remap([3,6], [1,1], 0)
    mapbiomas_forest = mapbiomas_forest.addBands(forest.rename(ee.String(year)))

mapbiomas_forest = mapbiomas_forest.select(mapbiomas_forest.bandNames().slice(1)).byte()

In [22]:
 '''
### Step 2 - Mapping the Annual Increment of Secondary Forests

Flags moments of regrowth and deforestation.

Intakes:
    mapbiomas_forest
    anthropic_mask
    w_mask
    urban_mask

Returns:
    regro -> Image with one band per year (1986 - 2020) and binary mask for REGROWTH EVENTS
    defor -> Image with one band per year (1986 - 2020) and binary mask for DEFORESTATION EVENTS
'''

regro = ee.Image()
defor = ee.Image()

for i in range(first_year, last_year):  # 1985-2019
    year1 = f'classification_{i}'
    year2 = f'classification_{i + 1}'
    # select the anthropic classifications for year 1 (starting 2015)
    # with the goal of keeping only forests regrowing on land of the desired previous LU types
    # (listed on LU_index)
    a_mask = anthropic_mask.select(year1)
    # Change pixel values to facilitate flagging of forest change (see sforest)
    forest1 = mapbiomas_forest.select(year1).remap([0, 1], [0, 2])
    forest2 = mapbiomas_forest.select(year2)
    # addition is 0 if was nonforest before and after; 1 if it was gained;
    # 2 if it was forest before and then was lost; 3 if it was forest in both.
    sforest = forest1.add(forest2).updateMask(w_mask).updateMask(urban_mask).updateMask(a_mask)
    for_gain = sforest.remap([0, 1, 2, 3], [0, 1, 0, 0]).rename(year2)
    for_loss = sforest.remap([0, 1, 2, 3], [0, 0, 1, 0]).rename(year2)
    regro = regro.addBands(for_gain)
    defor = defor.addBands(for_loss)

# remove the first band as that's all zeroes
regro = regro.select(regro.bandNames().slice(1)).byte()

In [23]:
'''
### Step 3 - Calculating and Mapping the Age of Secondary Forests

Intakes:
    regro -> Image with one band per year (1986 - 2020) and binary mask for REGROWTH EVENTS

Returns:
    extent -> Image with one band per year (1986 - 2020) and binary mask for CURRENTLY REGROWING forests
'''

extent = ee.Image()
# add pixels that became regrowing forests in 1986
extent = extent.addBands(regro.select('classification_1986').rename('classification_1986'))

for i in range(first_year + 1, last_year): #1986 to 2020
    year1 = f'classification_{i}'
    year2 = f'classification_{i + 1}'
    for_gain = regro.select(year2)
    # accumulated forest = pixels that gained forest in year1 + pixels that gained forest in year2
    acm_forest = extent.select(year1).add(for_gain)
    # mask by pixels that were shown to be forest in year2,
    # to eliminate any that may have regrown in 1986 but lost cover in 1987
    extent = extent.addBands(acm_forest.updateMask(mapbiomas_forest.select(year2)).rename(year2))

extent = extent.select(extent.bandNames().slice(1)).byte()

In [24]:
'''
### Step 4 - Calculating and Mapping the Age of Secondary Forests

Intakes:
    extent -> Image with one band per year (1986 - 2020) and binary mask for CURRENTLY REGROWING forests

Returns:
    ages -> Image with one band per year (1986 - 2020) with age of forest at every year
'''

ages = ee.Image()
ages = ages.addBands(extent.select('classification_1986').rename('classification_1986'))
ages = ages.slice(1) # remove "constant" band
age_total = ages # will use this as the "last total age" to keep iteratively adding values

for i in range(first_year + 1, last_year):
    year1 = f'classification_{i + 1}'# 1987-2020
    sforest = extent.select(year1) # forest cover in 1987
    age_total = age_total.add(sforest) # 1 year old forests in 1986 + cover in 1987
    f_year = mapbiomas_forest.select(year1)
    age_total = age_total.multiply(f_year) # mask by pixels that were forest that year, removing any forest loss
    ages = ages.addBands(age_total.rename(year1))

ages = ages.updateMask(ages) #keep only values as ages or NA

#ages range from 1 for those regrown in 2019-2020 to 35 for those regrown in 1985-1986
age = ages.select('classification_2020').rename('age')